In [1]:
from ns3gym import ns3env
from comet_ml import Experiment
import tqdm
import subprocess
from collections import deque

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
simTime = 10 # seconds
stepTime = 0.1  # seconds

EPISODE_COUNT = 3
steps_per_ep = int(simTime/stepTime)

SCRIPT_RUNNING = False

script_exec_command = f'../../waf --run "linear-mesh --simTime={simTime} --envStepTime={stepTime}"'
print("Steps per episode:", steps_per_ep)

Steps per episode: 100


In [3]:
port = 5555
seed = 0

simArgs = {"--simTime": simTime,
           "--testArg": 123,
           "--nodeNum": 5,
           "--distance": 500}
debug = False

subprocess.Popen(['bash', '-c', script_exec_command])
SCRIPT_RUNNING = True

env = ns3env.Ns3Env(port=port, stepTime=stepTime, startSim=0, simSeed=seed, simArgs=simArgs, debug=debug)

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


In [4]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

Observation space shape: Box(4,)
Action space shape: Box(1,)


In [5]:
experiment = Experiment(api_key="OZwyhJHyqzPZgHEpDFL1zxhyI",
                        project_name="rl-in-wifi", workspace="wwydmanski")

experiment.log_parameter("Episode count", EPISODE_COUNT)
experiment.log_parameter("Steps per episode", steps_per_ep)
 
for i in range(EPISODE_COUNT):
    if not SCRIPT_RUNNING:
        subprocess.Popen(['bash', '-c', script_exec_command])
        SCRIPT_RUNNING = True

    cumulative_reward = 0
    reward = 0

    obs = env.reset()
    with tqdm.trange(steps_per_ep) as t:
        for step in t:
            action = env.action_space.sample()
            action = action * 1 + 1
            obs, reward, done, info = env.step(action)
            cumulative_reward += reward
            experiment.log_metric("Round reward", reward, step=i*steps_per_ep+step)
            experiment.log_metric("Per-ep reward", cumulative_reward, step=i*steps_per_ep+step)
            t.set_postfix(cumul_rew=f"{cumulative_reward:.2f} Mb/s")

    experiment.log_metric("Cumulative reward", cumulative_reward, step=i)
    print(f"Sent {cumulative_reward:.2f} Mb/s.\tMean per round: {cumulative_reward} Episode {i+1}/{EPISODE_COUNT} finished\n")
    SCRIPT_RUNNING = False
            
experiment.end()

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/rl-in-wifi/46a5bae930d74bb59c3e596f876d12b3

100%|████████████████████████████████████████████████| 100/100 [00:40<00:00,  2.44it/s, cumul_rew=308.80 Mb/s, step=99]


Sent 308.80 Mb/s.	Mean per round: 308.798592209816 Episode 1/3 finished

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|███████████████████████████████████████████████| 100/100 [00:41<00:00,  2.42it/s, cumul_rew=320.91 Mb/s, step=199]


Sent 320.91 Mb/s.	Mean per round: 320.90879940986633 Episode 2/3 finished

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|███████████████████████████████████████████████| 100/100 [00:41<00:00,  2.41it/s, cumul_rew=319.91 Mb/s, step=299]


Sent 319.91 Mb/s.	Mean per round: 319.91327917575836 Episode 3/3 finished



COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Metrics:
COMET INFO:             Cumulative reward: 319.91327917575836
COMET INFO:                 Per-ep reward: 319.91327917575836
COMET INFO:                  Round reward: 3.935231924057007
COMET INFO:         sys.gpu.0.free_memory: 9519456256
COMET INFO:     sys.gpu.0.gpu_utilization: 5
COMET INFO:        sys.gpu.0.total_memory: 11811160064
COMET INFO:         sys.gpu.0.used_memory: 2291703808
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Still uploading
